In [0]:
#Creating Tables variables
patients    = spark.table("workspace.default.patients")
conditions  = spark.table("workspace.default.conditions")
encounters  = spark.table("workspace.default.encounters")
medications = spark.table("workspace.default.medications")
procedures  = spark.table("workspace.default.procedures")
allergies   = spark.table("workspace.default.allergies")
providers   = spark.table("workspace.default.providers")



In [0]:
#Group patients by gender 
patients.groupBy("gender").count().show()




In [0]:
"""
Longitudinal patient history
    1.)Condition Diagnosis 
    2.)Provider Encounter
    3.)Medication Prescription
"""
from pyspark.sql.functions import col, lit, row_number, to_date
from pyspark.sql.window import Window

# Define a window for ordering patient events chronologically
windowSpec = Window.partitionBy("patient").orderBy("start")

# Standardize each table with consistent columns
conditions_df = conditions.select(
    col("patient"),
    col("start"),
    col("description").alias("event_description"),
    lit("condition").alias("event_type")
)

encounters_df = encounters.select(
    col("patient"),
    col("start"),
    col("encounterclass").alias("event_description"),
    lit("encounter").alias("event_type")
)

medications_df = medications.select(
    col("patient"),
    col("start"),
    col("description").alias("event_description"),
    lit("medication").alias("event_type")
)

# Combine all into a unified timeline
events_df = conditions_df.unionByName(encounters_df).unionByName(medications_df)

# Add row number for sequence tracking per patient
patient_history_df = events_df.withColumn(
    "event_order",
    row_number().over(windowSpec)
).select(
    col("patient"),
    to_date("start").alias("event_date"),
    "event_type",
    "event_description",
    "event_order"
).orderBy("patient", "event_order")

# Show results
patient_history_df.show(truncate=False)
